In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys, glob, os, imageio
import numpy as np
import matplotlib.pyplot as plt
import torch
import helper.canon_supervised_dataset as dset
import helper.post_processing as pp
import helper.utils as ut
import matplotlib
import helper.gan_helper_fun as gh
from skimage import exposure
matplotlib.rcParams.update({'font.size': 22})

# View synthetic noise
In this notebook, we load in our pretrained noise model, and show our synthetic noise vs. real noise from our camera.

## Load in saved model

In [3]:
device = 'cuda:0'
gpu = 0
base_folder = 'saved_models/'
generator = gh.load_generator2(base_folder + 'noise_generator', gpu).to(device)
generator.keep_track = True

generator device 0
current epoch 144


## Load in image 

In [20]:
#options: gan_gray, gan_color, natural 
dataset_arg = 'gan_color'
dataset_list_test = dset.get_dataset_noise_visualization(dataset_arg)

In [21]:
test_loader = torch.utils.data.DataLoader(dataset=dataset_list_test, 
                                           batch_size=1,
                                           shuffle=False) 

In [22]:
sample = dataset_list_test.__getitem__(1)

In [23]:
with torch.no_grad():
    clean_raw = gh.t32(sample['gt_label_nobias'].unsqueeze(0).to(device))
    real_noisy = sample['noisy_input'].cpu().detach().numpy().transpose(1,2,3,0)
    clean_patch = sample['gt_label_nobias'].cpu().detach().numpy().transpose(1,2,3,0)

    generator.indices = [10,10]
    gen_noisy = generator(clean_raw, False)
    gen_noisy_np = gen_noisy.cpu().detach().numpy().transpose(0,2,3,1)

In [ ]:
ind = 9
sz = 256
pp = [200,200]

to_plot = [clean_patch[ind], gen_noisy_np[ind], real_noisy[ind]]
titles = ['clean patch', 'real noise', 'generated noise']

fig, ax = plt.subplots(1,3, figsize = (20,10))
for i in range(0,3):
    ax[i].imshow(to_plot[i][pp[0]:pp[0]+sz,pp[1]:pp[1]+sz,0:3]**(1/2.2)); 
    ax[i].axis('off');
    ax[i].set_title(titles[i]);


## View each noise component separately

In [ ]:
c = 0
all_noise_comps = list(generator.all_noise.keys())
fig, ax = plt.subplots(1,len(all_noise_comps), figsize = (20,10))
for i in range(0,len(all_noise_comps)):
        noise_curr = generator.all_noise[all_noise_comps[i]][0].transpose(1,2,0)[...,0:3]
        ax[i].imshow(noise_curr[...,c])
        ax[i].set_title(all_noise_comps[i])
        ax[i].axis('off')